Production planning problem from book Operations Research - A Model-Baesed Approach 3rd Edition by H.A Eiselt and Carl-Louis Sandblom


In [1]:
import linopy as lp
import pandas as pd
import xarray as xr

products = pd.Index(['P1', 'P2', 'P3'], name='Products')
machines = pd.Index(['M1', 'M2'], name='Machines')

processing_times = xr.DataArray([[3, 5, 4], [6, 1, 3]], coords=[machines, products])

processing_times

<xarray.DataArray (Machines: 2, Products: 3)> Size: 48B
array([[3, 5, 4],
       [6, 1, 3]])
Coordinates:
  * Machines  (Machines) object 16B 'M1' 'M2'
  * Products  (Products) object 24B 'P1' 'P2' 'P3'

In [2]:
machines_availability = xr.DataArray([540, 480], coords=[machines])

In [3]:
machines_availability

<xarray.DataArray (Machines: 2)> Size: 16B
array([540, 480])
Coordinates:
  * Machines  (Machines) object 16B 'M1' 'M2'

In [4]:
products_profit = xr.DataArray([5, 3.5, 4.5], coords=[products])
products_profit

<xarray.DataArray (Products: 3)> Size: 24B
array([5. , 3.5, 4.5])
Coordinates:
  * Products  (Products) object 24B 'P1' 'P2' 'P3'

In [5]:
linopy_model = lp.Model()

production = linopy_model.add_variables(lower=0, coords=[products], name='x')

machine_usage = production * processing_times
linopy_model.add_constraints(machine_usage.sum('Products') <= machines_availability)

total_profit_per_product = products_profit * production
linopy_model.add_objective(total_profit_per_product.sum(), sense='max')
linopy_model

Linopy LP model

Variables:
----------
 * x (Products)

Constraints:
------------
 * con0 (Machines)

Status:
-------
initialized

In [6]:
linopy_model.solve(io_api='direct')

Set parameter ComputeServer to value "http://server.grb-dev.int.aws.zooplus.io"
Set parameter ServerPassword
Set parameter CSAppName to value "delivery/transport-optimizer/dev"
Compute Server job ID: da1c81d5-3321-4571-a78d-dcfbfed21184
Capacity available on 'server.grb-dev.int.aws.zooplus.io' - connecting...
Established HTTP unencrypted connection
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.4.0 24E248)
Gurobi Compute Server Worker version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xe362fb62
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [4e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 5e+02]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns,

('ok', 'optimal')

In [14]:
linopy_model.solution

<xarray.Dataset> Size: 48B
Dimensions:   (Products: 3)
Coordinates:
  * Products  (Products) object 24B 'P1' 'P2' 'P3'
Data variables:
    x         (Products) float64 24B 20.0 0.0 120.0